<a href="https://colab.research.google.com/github/Ehrikr/Anamnese/blob/main/Prot%C3%B3tipo/Plantao24h.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
import google.generativeai as genai
import json
import os
import random
import time
from enum import Enum
# Para formatar melhor a saída de texto longas da IA
import textwrap
# Para buscar a API Key de forma segura
from google.colab import userdata

In [17]:
# Carrega a API Key do Secrets do Colab
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # Use o nome do secret que você criou
    if not GOOGLE_API_KEY:
        raise ValueError("API Key não encontrada no Colab Secrets. Por favor, configure-a.")

    genai.configure(api_key=GOOGLE_API_KEY)

    # Configurações do Modelo Gemini
    generative_model_config = {
        "temperature": 0.8, # Aumentei um pouco para mais variedade criativa
        "top_p": 0.95,
        "max_output_tokens": 3072,
    }
    # Configurações de segurança (ajuste conforme necessário)
    # BLOCK_NONE é permissivo. Se tiver problemas, pode precisar ser mais restritivo.
    safety_settings_permissive = [
        {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    ]

    generative_model = genai.GenerativeModel(
        model_name="gemini-2.0-flash", # Modelo rápido e capaz
        generation_config=genai.types.GenerationConfig(**generative_model_config),
        safety_settings=safety_settings_permissive
    )
    print("✅ SDK do Gemini configurada com sucesso usando a chave do Colab Secrets.")
except Exception as e:
    print(f"❌ Erro crítico ao configurar o SDK do Gemini: {e}")
    print("O jogo não poderá funcionar sem a API Gemini devidamente configurada.")
    generative_model = None

✅ SDK do Gemini configurada com sucesso usando a chave do Colab Secrets.


In [ ]:
# Enumerações e Constantes do Jogo (ClassesMedico, TracosMedico, COMBINACOES_ESPECIAIS, OpcoesDialogo, NPC_DATA)
# (Copie e cole as definições dessas classes e dicionários do código original que você forneceu aqui)
# Vou incluir uma versão resumida para manter o exemplo conciso, mas você deve usar as suas completas.

class ClassesMedico(Enum):
    CLINICO_GERAL = {"nome": "Clínico Geral", "habilidade": "Diagnóstico Rápido", "descricao_habilidade": "Revela 1 sintoma real oculto.", "exemplo_uso": "Nota febre persistente."}
    ENFERMEIRO = {"nome": "Enfermeiro(a)", "habilidade": "Triagem Afiada", "descricao_habilidade": "Descarta 1 pista falsa.", "exemplo_uso": "Relato de unicórnios é irrelevante."}
    DERMATOLOGISTA = {"nome": "Dermatologista", "habilidade": "Olho Clínico", "descricao_habilidade": "Identifica doenças de pele com precisão.", "exemplo_uso": "Manchas são psoríase, não aliens."}
    CIRURGIAO = {"nome": "Cirurgião(ã)", "habilidade": "Mãos de Aço", "descricao_habilidade": "Permite cirurgias arriscadas.", "exemplo_uso": "Opera sem exames... tumor benigno. Sorte!"}
    PSICOLOGO = {"nome": "Psicólogo(a)", "habilidade": "Leitura Emocional", "descricao_habilidade": "Descobre traumas ocultos.", "exemplo_uso": "Paciente mentiu, medo de hospitais."}
    FARMACEUTICO = {"nome": "Farmacêutico(a)", "habilidade": "Interação Química", "descricao_habilidade": "Detecta conflitos medicamentosos.", "exemplo_uso": "Remédio causa alucinações. Bingo!"}
    PEDIATRA = {"nome": "Pediatra", "habilidade": "Voz Acalmante", "descricao_habilidade": "Reduz resistência de crianças/pais.", "exemplo_uso": "Mãe se acalma e conta que criança comeu tinta."}
    CARDIOLOGISTA = {"nome": "Cardiologista", "habilidade": "Ritmo Cardíaco", "descricao_habilidade": "Detecta problemas cardíacos sem equipamento.", "exemplo_uso": "ECG normal, mas sente sopro... embolia!"}

class TracosMedico(Enum):
    CINICO = {"nome": "Cínico", "descricao": "+Sarcasmo, NPCs reagem."}
    EMPATICO = {"nome": "Empático", "descricao": "+Cooperação, -precisão."}
    TEORICO_CONSPIRACAO = {"nome": "Teórico da Conspiração", "descricao": "Pistas absurdas, acertos acidentais."}
    PERFECCIONISTA = {"nome": "Perfeccionista", "descricao": "Exige 100% certeza, demora."}
    DESASTRADO = {"nome": "Desastrado", "descricao": "Erros hilários, ganha simpatia."}
    VICIADO_TRABALHO = {"nome": "Viciado em Trabalho", "descricao": "Ignora cansaço, -empatia."}
    CURIOSO = {"nome": "Curioso", "descricao": "Perguntas aleatórias revelam pistas."}
    ARTISTA_FRUSTRADO = {"nome": "Artista Frustrado", "descricao": "Desenha sintomas (descrição textual)."}

COMBINACOES_ESPECIAIS = {
    ("CIRURGIAO", "DESASTRADO"): {"interacao_unica": "Opera o joelho errado... mas descobre um tumor!"},
    ("PSICOLOGO", "TEORICO_CONSPIRACAO"): {"interacao_unica": "Paciente controlado por aliens... Você concorda! (Revela esquizofrenia)"},
    ("CLINICO_GERAL", "TEORICO_CONSPIRACAO"): {
        "interacao_unica": "Sintomas comuns são 'sinais de controle mental'.",
        "respostas_exemplo": [("Pragmático", "Checar virose... ou chip da CIA."), ("Absurdo", "Radiação de torre 5G!")],
        "reacao_paciente_absurdo": "Doutor, só comi batata estragada..."
    },
    # ... (adicione TODAS as suas outras combinações aqui)
     ("DERMATOLOGISTA", "ARTISTA_FRUSTRADO"): {
        "interacao_unica": "Desenha sintomas como obras de arte, confundindo todos.",
        "respostas_exemplo": [("Pragmático", "Essa micose parece um Van Gogh!"), ("Absurdo", "Vou chamar essa doença de 'Monstro das Pústulas'.")],
        "reacao_paciente_absurdo": "Isso é uma emergência ou uma exposição?"
    },
}


class OpcoesDialogo(Enum):
    CINICO = "Cínica"; EMPATICO = "Empática"; PRAGMATICO = "Pragmático"; CAUTELOSO = "Cauteloso"
    FILOSOFICO = "Filosófico"; ABSURDO = "Absurdo"; HEROI = "Herói"; DESASTRE_IMINENTE = "Desastre Iminente"

NPC_DATA = {
    "Dr. Howser": {"descricao": "Cínico, manca.", "falas_iconicas": ["Todo mundo mente.", "NÃO é lúpus!"], "personalidade_prompt": "Genial, cínico, sarcástico."},
    "Dra. Meridee Grey": {"descricao": "Passional, introspectiva.", "falas_iconicas": ["Escolha-me. Ame-me."], "personalidade_prompt": "Comentários como voice-overs dramáticos."},
    "Enf. Cristina": {"descricao": "Competitiva, focada.", "falas_iconicas": ["Alguem opere para eu operar!"], "personalidade_prompt": "Ambiciosa, brilhante, direta."},
    "Dr. McSteamy": {"descricao": "Charmoso, confiante.", "falas_iconicas": ["Precisa de cirurgia... ou café?"], "personalidade_prompt": "Cirurgião plástico charmoso e bom no que faz."}
}
print("✅ Definições básicas do jogo carregadas.")

In [ ]:
# --- Funções Auxiliares ---
def print_lentamente(texto, delay=0.01): # Mais rápido para Colab
    for char in texto:
        print(char, end='', flush=True)
        time.sleep(delay)
    print()

def solicitar_input_numerico(prompt_msg, min_val, max_val):
    while True:
        try:
            escolha_str = input(prompt_msg).strip()
            if not escolha_str: return None
            escolha = int(escolha_str)
            if min_val <= escolha <= max_val: return escolha
            else: print_lentamente(f"Inválido. Entre {min_val} e {max_val}.")
        except ValueError: print_lentamente("Inválido. Insira um número.")

SAVE_FILE = "rpg_medico_save_colab.json" # Nome do arquivo de save no Colab

def salvar_jogo(jogador, casos_concluidos_ids):
    # (Mesma função salvar_jogo do código original)
    # Lembre-se: este arquivo é temporário no ambiente Colab.
    estado_jogador = {
        "nome": jogador.nome, "classe": jogador.classe.name, "traco": jogador.traco.name,
        "reputacao": jogador.reputacao, "pontuacao_diagnostica": jogador.pontuacao_diagnostica,
        "fama_de_maluco": jogador.fama_de_maluco, "casos_resolvidos": jogador.casos_resolvidos,
    }
    save_data = {"jogador": estado_jogador, "casos_concluidos_ids": casos_concluidos_ids}
    try:
        with open(SAVE_FILE, "w") as f: json.dump(save_data, f, indent=4)
        print_lentamente(f"Jogo salvo em '{SAVE_FILE}' (temporário no Colab)!")
    except Exception as e: print_lentamente(f"Erro ao salvar o jogo: {e}")


def carregar_jogo():
    # (Mesma função carregar_jogo do código original)
    try:
        with open(SAVE_FILE, "r") as f: save_data = json.load(f)
        dados_j = save_data["jogador"]
        jogador = Jogador(dados_j["nome"], ClassesMedico[dados_j["classe"]], TracosMedico[dados_j["traco"]])
        jogador.reputacao = dados_j["reputacao"]; jogador.pontuacao_diagnostica = dados_j["pontuacao_diagnostica"]
        jogador.fama_de_maluco = dados_j["fama_de_maluco"]; jogador.casos_resolvidos = dados_j["casos_resolvidos"]
        print_lentamente(f"Jogo carregado de '{SAVE_FILE}'!")
        return jogador, save_data.get("casos_concluidos_ids", [])
    except FileNotFoundError: print_lentamente(f"Nenhum jogo salvo ('{SAVE_FILE}') encontrado.")
    except Exception as e: print_lentamente(f"Erro ao carregar o jogo: {e}")
    return None, []


# --- Classes do Jogo (Jogador, CasoClinico, GeminiAPI, JogoMedicoRPG) ---
# (Copie e cole as definições COMPLETAS dessas classes do código original aqui)
# Incluindo:
# class Jogador: ...
# class CasoClinico: ...
# class GeminiAPI: ...
# class JogoMedicoRPG: ...

# Exemplo resumido (USE O SEU CÓDIGO ORIGINAL COMPLETO AQUI)
class Jogador:
    def __init__(self, nome, classe_medico: ClassesMedico, traco_medico: TracosMedico):
        self.nome = nome
        self.classe = classe_medico
        self.traco = traco_medico
        self.reputacao = 50
        self.pontuacao_diagnostica = 0
        self.fama_de_maluco = 0
        self.casos_resolvidos = 0

    def __str__(self):
        return (f"Dr(a). {self.nome}\n"
                f"  Classe: {self.classe.value['nome']} ({self.classe.value['habilidade']})\n"
                f"  Traço: {self.traco.value['nome']} ({self.traco.value['descricao']})\n"
                f"  Reputação: {self.reputacao}, Diagnósticos Certos: {self.pontuacao_diagnostica}, Fama de Maluco: {self.fama_de_maluco}")

    def verificar_combinacao_especial(self):
        chave = (self.classe.name, self.traco.name)
        return COMBINACOES_ESPECIAIS.get(chave)

class CasoClinico:
    def __init__(self, id_caso, titulo_caso, descricao_inicial_paciente, paciente_info,
                 sintomas_reais_ocultos, pistas_falsas_criativas, diagnostico_real_medico,
                 doenca_explicacao_simplificada, erro_comum_relacionado, dica_vida_real,
                 introducao_narrativa_imersiva=None,
                 referencia_npc_howser=None, referencia_npc_meridee=None, **kwargs):

        # ... (atributos existentes como id_caso, titulo_caso, etc. permanecem) ...
        self.id_caso = id_caso
        self.titulo_caso = titulo_caso
        self.introducao_narrativa_imersiva = introducao_narrativa_imersiva if introducao_narrativa_imersiva else descricao_inicial_paciente
        self.descricao_inicial_paciente = descricao_inicial_paciente
        self.paciente_info = paciente_info
        self.sintomas_reais_ocultos = sintomas_reais_ocultos
        self.sintomas_revelados_jogador = list(self.paciente_info.get('sintomas_visiveis_gemini', []))
        self.pistas_falsas_criativas = pistas_falsas_criativas
        self.pistas_falsas_descartadas = []
        self.diagnostico_real_medico = diagnostico_real_medico
        self.doenca_explicacao_simplificada = doenca_explicacao_simplificada
        self.erro_comum_relacionado = erro_comum_relacionado
        self.dica_vida_real = dica_vida_real
        self.hipoteses_jogador = []
        self.exames_realizados = []
        self.progresso_narrativo_cena_atual = [] # Para dar contexto ao Gemini para as 8 opções
        self.resolvido = False
        self.desfecho_final = None
        self.dicas_howser_caso = [referencia_npc_howser] if referencia_npc_howser and isinstance(referencia_npc_howser, str) else []
        self.diario_meridee_caso = [referencia_npc_meridee] if referencia_npc_meridee and isinstance(referencia_npc_meridee, str) else []
        self.dados_extras_gemini = kwargs

        # SUBSTITUIR os atributos de opções anteriores por este:
        self.opcoes_cena_atuais = [] # Irá armazenar a lista de 8 dicionários {"tom": ..., "opcao_texto": ...}

        # Manter para opções de diagnóstico final
        self.opcoes_diagnostico_atuais = []


    def adicionar_progresso_cena(self, texto_narrativo: str):
        """Adiciona à narrativa da cena atual e imprime."""
        if not isinstance(texto_narrativo, str): texto_narrativo = str(texto_narrativo)
        # Limita o tamanho do histórico recente para não sobrecarregar o prompt
        if len(self.progresso_narrativo_cena_atual) > 5:
            self.progresso_narrativo_cena_atual.pop(0) # Remove o mais antigo
        self.progresso_narrativo_cena_atual.append(texto_narrativo)

        # Imprime a nova informação imediatamente
        wrapped_text = "\n".join(textwrap.wrap(texto_narrativo, width=90, replace_whitespace=False, drop_whitespace=False))
        print_lentamente(wrapped_text)

    def exibir_resumo_mental(self, jogador_nome): # SUBSTITUI exibir_informacoes_caso
        """Gera e exibe um resumo narrativo do que o jogador sabe (estilo Disco Elysium)."""
        # Este método pode ser chamado menos frequentemente ou como uma ação do jogador "Refletir".
        # Para o fluxo principal, a narrativa contínua é mais importante.
        # Por enquanto, vamos manter a impressão manual do resumo, mas o ideal seria gerar com IA.

        print_lentamente(f"\n--- Dr(a). {jogador_nome} reflete sobre o caso: {self.titulo_caso} ---")
        if self.sintomas_revelados_jogador:
            sintomas_str = ", ".join(list(set(self.sintomas_revelados_jogador)))
            print_lentamente(f"Pelo que pude apurar até agora, o paciente {self.paciente_info.get('nome')} apresenta: {sintomas_str}.")
        else:
            print_lentamente(f"Ainda estou tentando entender os sintomas principais de {self.paciente_info.get('nome')}.")

        if self.exames_realizados:
            print_lentamente("Resultados de exames já vistos:")
            for exame, resultado_narrativa in self.exames_realizados:
                resumo_resultado = resultado_narrativa.split('\n')[0]
                print_lentamente(f"- {exame}: {resumo_resultado}...")

        if self.hipoteses_jogador:
            hipoteses_str = "; ".join(self.hipoteses_jogador)
            print_lentamente(f"Minhas suspeitas atuais são: {hipoteses_str}.")
        print_lentamente("----------------------------------------------------")


    def resetar_opcoes_cena(self):
        self.opcoes_cena_atuais = []
        # self.progresso_narrativo_cena_atual = [] # Não resetar aqui, acumula dentro da cena. Resetar no início de uma nova "cena" maior.


class GeminiAPI:
    def __init__(self, model):
        self.model = model

    def _gerar_conteudo_seguro(self, prompt_text, is_json_output=False, max_retries=3):
        # ... (manter o método _gerar_conteudo_seguro como antes)
        if not self.model:
            print_lentamente("❌ Modelo Gemini não inicializado. Verifique a configuração da API Key.")
            return "Erro: Modelo Gemini não configurado."

        for attempt in range(max_retries):
            try:
                print_lentamente(f"... (API Gemini processando solicitação - Tentativa {attempt+1}/{max_retries}) ...")
                response = self.model.generate_content(prompt_text)

                if not response.candidates or not response.candidates[0].content.parts:
                    finish_reason_name = "DESCONHECIDO"
                    safety_ratings_text = "N/A"
                    if response.candidates and response.candidates[0].finish_reason:
                        finish_reason_name = response.candidates[0].finish_reason.name
                    if response.candidates and response.candidates[0].safety_ratings:
                         safety_ratings_text = "".join(f"\n  - {sr.category}: {sr.probability.name}" for sr in response.candidates[0].safety_ratings)

                    error_message = (f"API Gemini bloqueou ou não gerou conteúdo. "
                                     f"Razão: {finish_reason_name}. Safety Ratings: {safety_ratings_text}")
                    print_lentamente(f"⚠️ Aviso da API (tentativa {attempt+1}/{max_retries}): {error_message}")

                    if "SAFETY" in finish_reason_name or (response.prompt_feedback and response.prompt_feedback.block_reason == 'SAFETY'):
                        return f"Erro Crítico de Segurança da API: {error_message}. O prompt pode precisar de ajuste."

                    time.sleep(1.8 ** attempt)
                    continue

                generated_text = response.text.strip()

                if is_json_output:
                    if generated_text.startswith("```json"): generated_text = generated_text[7:]
                    elif generated_text.startswith("```"): generated_text = generated_text[3:]
                    if generated_text.endswith("```"): generated_text = generated_text[:-3]
                    generated_text = generated_text.strip()

                    try:
                        parsed_json = json.loads(generated_text)
                        return parsed_json
                    except json.JSONDecodeError as json_e:
                        print_lentamente(f"❌ Erro ao decodificar JSON da API (tentativa {attempt+1}): {json_e}.")
                        print_lentamente(f"Texto recebido (início): {generated_text[:600]}...")
                        if attempt == max_retries - 1:
                            return f"Erro Crítico de formato JSON: {generated_text[:600]}"
                        time.sleep(1.8 ** attempt)
                        continue
                return generated_text

            except Exception as e:
                print_lentamente(f"❌ Erro inesperado na chamada da API Gemini (tentativa {attempt+1}): {type(e).__name__} - {e}")
                if attempt == max_retries - 1:
                    return f"Erro Crítico: Falha na API Gemini após {max_retries} tentativas. ({type(e).__name__})"
                time.sleep(1.8 ** attempt)

        return f"Erro Crítico: Falha ao gerar conteúdo após {max_retries} tentativas (esgotado)."

    def gerar_caso_clinico(self, jogador: Jogador, tema_caso=None, dificuldade="normal", casos_anteriores_ids=None):
        # MODIFICAR ESTE PROMPT PARA O ESTILO NARRATIVO (Etapa 2)
        # Por agora, vamos manter o foco nas opções, mas adicionaremos o campo introducao_narrativa_imersiva
        if casos_anteriores_ids is None: casos_anteriores_ids = []
        prompt_base = f"""
        Você é um roteirista de "Plantão 24h: O Mistério do Paciente X".
        Jogador: Dr(a). {jogador.nome}, {jogador.classe.value['nome']} com traço "{jogador.traco.value['nome']}".
        Hospital decadente, anos 80. Tom: drama médico, educativo, humor, absurdo.
        Reputação: {jogador.reputacao}, Pontos Diag: {jogador.pontuacao_diagnostica}, Fama Maluco: {jogador.fama_de_maluco}.
        Não repita IDs: {', '.join(map(str, casos_anteriores_ids)) if casos_anteriores_ids else 'Nenhum'}.
        {f"Tema Sugerido: {tema_caso}" if tema_caso else ""} Dificuldade: {dificuldade}.

        Gere um caso clínico NOVO e ÚNICO em formato JSON ESTRITO com os seguintes campos:
        "id_caso", "titulo_caso",
        "introducao_narrativa_imersiva" (string, 2-3 parágrafos descrevendo a chegada do paciente, ambiente, sintomas iniciais e histórico breve de forma orgânica, estilo Disco Elysium),
        "descricao_inicial_paciente" (string, resumo factual da situação para referência interna, pode ser similar à introdução mas mais direto),
        "paciente_info": {{"nome", "idade", "historico_breve" /*ESTE HISTÓRICO BREVE DEVE ESTAR NA INTRODUÇÃO NARRATIVA*/, "sintomas_visiveis_gemini": [/*Sintomas que ficam óbvios pela introdução*/]}},
        "sintomas_reais_ocultos": ["sint_oculto1"], "pistas_falsas_criativas": ["pista_falsa1"],
        "diagnostico_real_medico", "doenca_explicacao_simplificada", "erro_comum_relacionado", "dica_vida_real",
        "dificuldade_estimada",
        "potenciais_interacoes_classe_traco",
        "referencia_npc_howser" (opcional), "referencia_npc_meridee" (opcional).

        ATENÇÃO: O JSON DEVE SER VÁLIDO.
        Exemplo para "introducao_narrativa_imersiva":
        "O cheiro de antisséptico barato e desespero mal disfarçado pairava no ar da emergência do St. Jude's como uma mortalha invisível. Dr(a). {jogador.nome} suspirou, o café requentado fazendo pouco para afastar o cansaço da noite anterior. Foi quando as portas automáticas se abriram com um chiado asmático, revelando um homem cambaleante, Sr. Bartholomew Quibble, uns 30 e poucos anos, pele de um tom laranja-abóbora vibrante que faria um agente de trânsito sentir inveja. 'Doutor... acho que comi algo... estranho', ele murmurou, os olhos arregalados e inquietos vasculhando a sala. Ele mencionou ser artista de circo, com uma 'noite agitada' e um 'lanche misterioso'. Sua confusão era palpável."
        (O campo "sintomas_visiveis_gemini" para este exemplo seria: ["Pele laranja-abóbora vibrante", "Confusão mental", "Inquietação"])
        (O campo "descricao_inicial_paciente" seria um resumo: "Homem de 37 anos, artista de circo, chega com pele laranja, confusão e inquietação após noite agitada e lanche misterioso.")
        """
        return self._gerar_conteudo_seguro(prompt_base, is_json_output=True)


    def gerar_opcoes_de_cena(self, jogador: Jogador, caso: CasoClinico, historico_recente_cena: list):
        sintomas_conhecidos_str = ", ".join(list(set(caso.sintomas_revelados_jogador))) if caso.sintomas_revelados_jogador else "pouco se sabe ainda"
        exames_feitos_str = ", ".join([ex[0] for ex in caso.exames_realizados]) if caso.exames_realizados else "nenhum exame realizado"
        hipoteses_str = ", ".join(caso.hipoteses_jogador) if caso.hipoteses_jogador else "nenhuma hipótese clara"

        # Pega as últimas 2-3 interações para dar contexto recente
        contexto_recente_str = "\n".join(historico_recente_cena[-3:]) if historico_recente_cena else "Início da cena."

        prompt = f"""
        Você é o mestre do jogo RPG médico "Plantão 24h: O Mistério do Paciente X", com um estilo narrativo inspirado em Disco Elysium.
        Jogador: Dr(a). {jogador.nome} (Classe: {jogador.classe.value['nome']}, Traço: {jogador.traco.value['nome']}).
        Caso Atual: '{caso.titulo_caso}' - Paciente: {caso.paciente_info.get('nome')}.
        Situação Resumida: Sintomas conhecidos: {sintomas_conhecidos_str}. Exames feitos: {exames_feitos_str}. Hipóteses do jogador: {hipoteses_str}.
        Últimos eventos na cena: {contexto_recente_str}
        Diagnóstico Real (SEU CONHECIMENTO APENAS, NÃO REVELE AO JOGADOR NAS OPÇÕES): {caso.diagnostico_real_medico}.

        Gere EXATAMENTE 8 opções de ação/diálogo que o Dr(a). {jogador.nome} pode tomar AGORA.
        Cada opção deve corresponder a um dos seguintes tons distintos e ser uma ação/fala concreta e interessante:
        1.  **Cínica:** Exemplo: "Claro, essa sua 'dor excruciante' melhora na hora de receber alta, né?"
        2.  **Empática:** Exemplo: "Vejo que você está sofrendo. Vamos encontrar juntos a causa disso, conte comigo."
        3.  **Pragmática:** Exemplo: "Precisamos de dados. Vou solicitar um hemograma e um raio-X de tórax imediatamente."
        4.  **Cautelosa:** Exemplo: "Antes de qualquer conclusão, quero observar a evolução dos sintomas por mais algumas horas e refazer aquele exame."
        5.  **Filosófica:** Exemplo: "Essa erupção na pele... seria um mapa das suas angústias internas ou apenas uma dermatite comum? O que é 'comum', afinal?"
        6.  **Absurda:** Exemplo: "Tenho uma teoria: isso tudo é obra de gnomos radiologistas infiltrados no sistema de ventilação!"
        7.  **Heróica:** Exemplo: "Não se preocupe! Com a minha experiência e dedicação, vamos vencer essa doença misteriosa, custe o que custar!"
        8.  **Desastre Iminente:** Exemplo: "Vou tentar aquela manobra experimental que vi num filme B... O que pode dar errado?"

        As opções podem incluir:
        - Fazer perguntas específicas ao paciente.
        - Realizar uma observação ou exame físico detalhado.
        - Sugerir um exame complementar específico.
        - Tomar uma decisão sobre o próximo passo do tratamento (se aplicável).
        - Expressar um pensamento ou teoria (que pode ou não ser útil).
        - Realizar uma ação que possa ter consequências diretas (boas ou ruins).

        Considere a CLASSE '{jogador.classe.value['nome']}' e o TRAÇO '{jogador.traco.value['nome']}' do jogador ao formular as opções, tornando-as mais personalizadas dentro de cada tom.
        Por exemplo, um Cirurgião Pragmático pode sugerir "Preparar para cirurgia exploratória", enquanto um Clínico Pragmático pode dizer "Vamos começar com os exames básicos e ver onde isso nos leva".
        Um Psicólogo Empático pode sugerir "Explorar como esses sintomas estão afetando seu estado emocional".

        Retorne as opções como uma lista JSON de 8 dicionários. Cada dicionário deve ter as chaves "tom" (string, o nome do tom ex: "Cínica") e "opcao_texto" (string, o texto da ação/diálogo).
        A ordem dos tons na lista deve ser a mesma listada acima (Cínica, Empática, etc.).

        Exemplo de formato de retorno:
        [
            {{"tom": "Cínica", "opcao_texto": "Então, essa 'quase morte' foi antes ou depois do seu terceiro prato de feijoada?"}},
            {{"tom": "Empática", "opcao_texto": "Respire fundo, estou aqui para ajudar. Pode me contar exatamente como se sente?"}},
            {{"tom": "Pragmática", "opcao_texto": "Vamos direto aos fatos. Preciso de uma amostra de sangue para análise agora."}},
            {{"tom": "Cautelosa", "opcao_texto": "Não quero me precipitar. Vamos monitorar seus sinais vitais por uma hora antes de decidir."}},
            {{"tom": "Filosófica", "opcao_texto": "O que é a dor, senão um lembrete da nossa frágil existência? Mas, falando em existência, quando isso começou?"}},
            {{"tom": "Absurda", "opcao_texto": "Suspeito de influência lunar. Você notou algo estranho durante a última lua cheia... como o desejo incontrolável de uivar?"}},
            {{"tom": "Heróica", "opcao_texto": "Não tema, cidadão! Com o poder da medicina moderna e minha intuição infalível, desvendaremos este mistério!"}},
            {{"tom": "Desastre Iminente", "opcao_texto": "Acho que vou testar aquele coquetel de medicamentos vencidos que encontrei no fundo da gaveta. Para fins científicos, claro."}}
        ]
        """
        return self._gerar_conteudo_seguro(prompt, is_json_output=True)

    def gerar_resposta_interativa(self, jogador: Jogador, caso: CasoClinico, acao_escolhida_texto: str, tom_da_acao: str):
        sintomas_conhecidos_str = ", ".join(list(set(caso.sintomas_revelados_jogador))) if caso.sintomas_revelados_jogador else "pouco claro"

        prompt_interacao = f"""
        Mestre do jogo "Plantão 24h" (estilo Disco Elysium).
        Jogador: Dr(a). {jogador.nome} (Classe: {jogador.classe.value['nome']}, Traço: {jogador.traco.value['nome']}).
        Caso: '{caso.titulo_caso}' - Paciente: {caso.paciente_info.get('nome')}.
        Sintomas conhecidos pelo jogador: {sintomas_conhecidos_str}.
        (Seu conhecimento APENAS: Sintomas Reais Ocultos que o jogador ainda não descobriu: {', '.join(s for s in caso.sintomas_reais_ocultos if s not in caso.sintomas_revelados_jogador)}, Diag.Real: {caso.diagnostico_real_medico})

        O jogador escolheu a seguinte AÇÃO/DIÁLOGO, imbuída de um tom '{tom_da_acao}':
        "{acao_escolhida_texto}"

        Descreva o RESULTADO NARRATIVO desta ação/diálogo (2-5 frases).
        Mantenha o tom do jogo (anos 80, hospital decadente, humor negro, drama, introspecção).
        A narrativa deve fluir, como se fosse uma continuação da cena.
        -   Se a ação foi uma pergunta/exame físico: Descreva a reação do paciente, o que o jogador observa, e qualquer nova informação ou sintoma revelado. Se um sintoma for revelado DIRETAMENTE pela fala do paciente, coloque-o entre aspas simples na narrativa (ex: O paciente aperta os olhos e confessa: 'Doutor, para ser honesto, também sinto uma tontura estranha desde ontem.')
        -   Se a ação sugeriu um exame complementar (ex: uma opção "Pragmática" foi "Pedir um Raio-X"): Descreva a cena e APRESENTE O RESULTADO DO EXAME de forma narrativa e interessante.
        -   Se a ação foi mais um comentário ou uma decisão com consequências (especialmente "Desastre Iminente" ou "Heróica"): Descreva as consequências diretas.
        Considere o tom '{tom_da_acao}' ao descrever a reação do mundo ou o resultado. Uma ação "Cínica" pode irritar o paciente. Uma "Desastre Iminente" deve levar a um problema (geralmente cômico ou que complica o caso). Uma "Heróica" pode inspirar ou ser vista como exagero.
        Seja criativo e faça a história progredir! A resposta DEVE SER APENAS A NARRATIVA DO RESULTADO.
        """
        return self._gerar_conteudo_seguro(prompt_interacao)

    # Manter os outros métodos da GeminiAPI (gerar_caso_clinico, gerar_opcoes_diagnostico_final, etc.)
    # O prompt de gerar_caso_clinico já está ajustado para pedir "introducao_narrativa_imersiva".
    # O prompt de gerar_dialogo_npc (para intervenções aleatórias) também está ok.



    def gerar_opcoes_diagnostico_final(self, jogador: Jogador, caso: CasoClinico):
        sintomas_conhecidos_str = ", ".join(list(set(caso.sintomas_revelados_jogador)))

        prompt = f"""
        No jogo "Plantão 24h", Dr(a). {jogador.nome} ({jogador.classe.value['nome']}, traço: {jogador.traco.value['nome']}) está prestes a dar o diagnóstico final para o caso '{caso.titulo_caso}'.
        Paciente: {caso.paciente_info.get('nome')}. Sintomas conhecidos: {sintomas_conhecidos_str}.
        O DIAGNÓSTICO CORRETO para este caso é: "{caso.diagnostico_real_medico}".

        Gere uma lista JSON com 6 OPÇÕES DE DIAGNÓSTICO.
        1.  Uma das opções DEVE SER EXATAMENTE o diagnóstico correto: "{caso.diagnostico_real_medico}".
        2.  As outras 5 opções devem ser diagnósticos INCORRETOS. Estes devem ser:
            * Plausíveis, considerando alguns dos sintomas apresentados.
            * Ou tematicamente interessantes/divertidos/bizarros, especialmente se o traço do jogador for Teórico da Conspiração ou Desastrado, ou se o caso tiver elementos estranhos.
            * Não torne os errados ridiculamente óbvios se o caso tiver um tom mais sério, a menos que o objetivo seja humorístico.
            * Tente variar a natureza dos diagnósticos errados (ex: uma síndrome rara, uma condição comum mas errada para os sintomas chave, uma interpretação errada de um sintoma).
        3.  A ordem das 6 opções na lista JSON deve ser aleatória.

        Retorne APENAS a lista JSON de strings. Exemplo (o correto aqui é "Apendicite Aguda"):
        [
            "Gastroenterite Viral Aguda",
            "Apendicite Aguda",
            "Síndrome do Intestino Irritável com exacerbação",
            "Cólica Renal Direita",
            "Infecção Urinária Alta (Pielonefrite)",
            "Obstrução Intestinal por Ingestão de Lego (se o paciente for criança ou Desastrado)"
        ]
        """
        return self._gerar_conteudo_seguro(prompt, is_json_output=True)

    # Os métodos gerar_dialogo_npc, gerar_resposta_interativa, gerar_desfecho_caso, gerar_imagem_sintoma_descricao_textual
    # permanecem como no seu código original, mas lembre-se de que os prompts deles podem precisar de ajustes
    # para se alinharem com o novo estilo narrativo e as opções pré-definidas.
    # Por exemplo, gerar_resposta_interativa agora recebe uma *ação escolhida de uma lista*, não texto livre.
    def gerar_resposta_interativa(self, jogador: Jogador, caso: CasoClinico, acao_escolhida: str, tipo_acao: str): # tipo_acao: "investigacao" ou "exame"
        # O prompt aqui precisa ser ajustado para refletir que 'acao_escolhida' é uma das opções geradas.
        sintomas_conhecidos_str = ", ".join(list(set(caso.sintomas_revelados_jogador))) if caso.sintomas_revelados_jogador else "nenhum sintoma claro ainda"

        prompt_interacao = f"""
        Mestre de "Plantão 24h". Jogador: Dr(a). {jogador.nome} ({jogador.classe.value['nome']}, {jogador.traco.value['nome']}). Rep: {jogador.reputacao}, FamaMaluco: {jogador.fama_de_maluco}.
        Caso: {caso.id_caso} - {caso.paciente_info.get('nome', 'Paciente')}.
        Sintomas conhecidos pelo jogador: {sintomas_conhecidos_str}.
        (Seu conhecimento: Sintomas Reais Ocultos que o jogador ainda não descobriu: {', '.join(s for s in caso.sintomas_reais_ocultos if s not in caso.sintomas_revelados_jogador)}, Diag.Real: {caso.diagnostico_real_medico})

        O jogador escolheu a seguinte AÇÃO de {tipo_acao}: "{acao_escolhida}"

        Gere a NARRAÇÃO do resultado desta ação (2-5 frases). Mantenha o tom (anos 80, humor, drama, estilo Disco Elysium com descrições vívidas e diálogos).
        A resposta DEVE SER APENAS A NARRATIVA DO RESULTADO DA AÇÃO.
        - Se a ação foi uma pergunta/exame físico: Descreva a reação do paciente, o que o jogador observa, e qualquer nova informação ou sintoma revelado. Se um sintoma for revelado DIRETAMENTE pela fala do paciente, coloque-o entre aspas simples na narrativa (ex: O paciente geme: 'Ah, doutor, também sinto uma pontada forte aqui no peito quando respiro fundo.')
        - Se a ação foi solicitar um exame: Descreva a cena da realização do exame (se relevante) e APRESENTE O RESULTADO DO EXAME de forma narrativa. Ex: "A enfermeira volta com o resultado do Hemograma. Você analisa os números: Leucócitos em 18.000, com desvio à esquerda. Claramente há uma infecção em curso." ou "O Raio-X de tórax não mostra nada além de um coração um pouco grande demais para o gosto do Dr. Howser, que resmunga algo sobre 'excesso de sentimentalismo'."
        Considere a classe/traço do jogador para nuances na descrição ou na reação dos outros.
        Seja criativo!
        """
        return self._gerar_conteudo_seguro(prompt_interacao)

    # Manter os outros métodos da GeminiAPI (gerar_dialogo_npc, gerar_desfecho_caso, gerar_imagem_sintoma_descricao_textual)
    # Lembre-se de que o prompt de gerar_dialogo_npc para NPCs aleatórios precisará ser contextual.
    def gerar_dialogo_npc(self, npc_nome, situacao_atual, jogador: Jogador, historico_conversa=None):
        npc_info = NPC_DATA.get(npc_nome)
        if not npc_info: return "NPC desconhecido."
        # (MANTENHA SEU PROMPT DETALHADO ORIGINAL PARA gerar_dialogo_npc AQUI)
        prompt_dialogo = f"""
        Você é {npc_nome} ({npc_info['descricao']}) em "Plantão 24h".
        {npc_info['personalidade_prompt']}
        Jogador: Dr(a). {jogador.nome} ({jogador.classe.value['nome']}, traço: {jogador.traco.value['nome']}).
        Situação atual no caso '{situacao_atual.get('titulo_caso','desconhecido')}': {situacao_atual.get('resumo_situacao_para_npc','O jogador está investigando.')}
        Gere uma fala CURTA (1-2 frases) e característica do {npc_nome} para esta situação. Pode ser um comentário, uma dica velada, ou uma observação sarcástica/dramática.
        Considere falas icônicas: {', '.join(npc_info['falas_iconicas'])}.
        """
        return self._gerar_conteudo_seguro(prompt_dialogo)

    def gerar_desfecho_caso(self, jogador: Jogador, caso: CasoClinico, diagnostico_jogador: str):
        # (Mantenha o método gerar_desfecho_caso como no seu código original, com a lógica de acerto e o prompt detalhado para o JSON do desfecho)
        # A lógica de acerto já foi melhorada.
        diag_j_lower = diagnostico_jogador.lower().strip()
        diag_r_lower = caso.diagnostico_real_medico.lower().strip()

        acertou = False
        if diag_j_lower == diag_r_lower: acertou = True
        elif diag_j_lower in diag_r_lower or diag_r_lower in diag_j_lower: acertou = True
        else:
            palavras_j = set(diag_j_lower.split())
            palavras_r = set(diag_r_lower.split())
            intersecao = palavras_j.intersection(palavras_r)
            uniao = palavras_j.union(palavras_r)
            if uniao and (len(intersecao) / len(uniao) > 0.6 or (len(intersecao) >=2 and len(diag_j_lower)>5)):
                acertou = True

        base_rep_acerto = 10; base_rep_erro_leve = -5; base_rep_erro_grave = -10
        dificuldade_caso = caso.dados_extras_gemini.get("dificuldade_estimada", "normal")

        if acertou:
            if dificuldade_caso == "difícil": base_rep_acerto = 15
            elif dificuldade_caso == "bizarro": base_rep_acerto = 20
            elif dificuldade_caso == "fácil": base_rep_acerto = 7
            rep_delta = base_rep_acerto
        else:
            if dificuldade_caso == "difícil": base_rep_erro_grave = -15
            elif dificuldade_caso == "bizarro": base_rep_erro_grave = -20
            elif dificuldade_caso == "fácil": base_rep_erro_leve = -3; base_rep_erro_grave = -7
            rep_delta = base_rep_erro_grave

        prompt_desfecho = f"""
        Mestre de "Plantão 24h". Jogador: Dr(a). {jogador.nome} ({jogador.classe.value['nome']}, {jogador.traco.value['nome']}).
        Caso: {caso.id_caso}. Diag.Real: {caso.diagnostico_real_medico}. Jogador diagnosticou: "{diagnostico_jogador}".
        O jogador acertou o diagnóstico? {"Sim" if acertou else "Não"}.

        Gere o DESFECHO NARRATIVO e a FICHA EDUCATIVA em JSON ESTRITO.
        Desfecho: Consequências do diagnóstico. Considere traço/classe. Se o jogador for Teórico da Conspiração e o diagnóstico for absurdo mas ACERTAR um caso bizarro, crie uma cena memorável. Se ERRAR com teoria da conspiração, as consequências podem ser piores ou mais cômicas.

        JSON ESTRITO com os campos:
        "desfecho_narrativa" (string),
        "diagnostico_final_jogador": "{diagnostico_jogador}",
        "diagnostico_correto_caso": "{caso.diagnostico_real_medico}",
        "acertou": {acertou},
        "ficha_educativa_titulo": "Ficha Educativa: {caso.titulo_caso}",
        "doenca_real_ficha": "{caso.diagnostico_real_medico}",
        "explicacao_doenca_ficha": "{caso.doenca_explicacao_simplificada}",
        "erro_comum_ficha": "{caso.erro_comum_relacionado}",
        "dica_vida_real_ficha": "{caso.dica_vida_real}",
        "comentario_howser_final" (string opcional),
        "comentario_meridee_final" (string opcional),
        "reputacao_delta" (int: use {rep_delta} como base, mas pode ajustar +/-2 com base na criatividade/gravidade do acerto/erro narrado),
        "pontuacao_diagnostica_delta" (int: +1 se acertou, 0 se errou),
        "fama_de_maluco_delta" (int: +0 a +5 se ação/diag bizarro, mesmo que acerte. +0 se normal).

        ATENÇÃO: O JSON DEVE SER VÁLIDO. Envolva todas as chaves e strings do JSON com aspas duplas.
        """
        return self._gerar_conteudo_seguro(prompt_desfecho, is_json_output=True)

    def gerar_imagem_sintoma_descricao_textual(self, jogador: Jogador, sintoma_para_desenhar: str):
        prompt_desenho = f"""
        Dr(a). {jogador.nome} ({jogador.classe.value['nome']}, Artista Frustrado) desenhou: "{sintoma_para_desenhar}".
        Descreva, de forma cômica/exagerada, como um artista incompreendido, esse desenho (1-2 frases).
        Ex: Sintoma: "Rash cutâneo". Descrição: "Behold! 'A Agonia Rubra'! Pápulas como luas sangrentas! Ou só bolhas. Mas com alma!"
        """
        return self._gerar_conteudo_seguro(prompt_desenho)


class JogoMedicoRPG:
    def __init__(self):
        self.jogador = None
        self.gemini_api = GeminiAPI(generative_model) # Usa o modelo global configurado
        self.caso_atual = None
        self.casos_concluidos_ids = []

    def _aplicar_habilidades_inicio_caso(self):
        if not self.caso_atual or not self.jogador : return
        if self.jogador.classe == ClassesMedico.CLINICO_GERAL and self.caso_atual.sintomas_reais_ocultos:
            sintomas_nao_revelados_ainda = [s for s in self.caso_atual.sintomas_reais_ocultos if s not in self.caso_atual.sintomas_revelados_jogador]
            if sintomas_nao_revelados_ainda:
                sintoma_revelado_habilidade = random.choice(sintomas_nao_revelados_ainda)
                self.caso_atual.sintomas_revelados_jogador.append(sintoma_revelado_habilidade)
                msg_habilidade = f"💡 ({self.jogador.classe.value['habilidade']}) Você tem um palpite... parece que o paciente também apresenta '{sintoma_revelado_habilidade}', embora não tenha mencionado claramente."
                self.caso_atual.adicionar_progresso_cena(msg_habilidade)
        elif self.jogador.classe == ClassesMedico.ENFERMEIRO and self.caso_atual.pistas_falsas_criativas:
            pistas_ativas_ainda = [p for p in self.caso_atual.pistas_falsas_criativas if p not in self.caso_atual.pistas_falsas_descartadas]
            if pistas_ativas_ainda:
                pista_descartada_habilidade = random.choice(pistas_ativas_ainda)
                self.caso_atual.pistas_falsas_descartadas.append(pista_descartada_habilidade)
                msg_habilidade = f"🎯 ({self.jogador.classe.value['habilidade']}) Sua intuição de enfermagem apita! A informação sobre '{pista_descartada_habilidade}' parece ser um completo alarme falso."
                self.caso_atual.adicionar_progresso_cena(msg_habilidade)

    def iniciar_novo_jogo(self):
        print_lentamente("Bem-vindo ao Plantão 24h: O Mistério do Paciente X!")
        print_lentamente("Anos 80. Hospital decadente. Você é a nova esperança (ou desastre ambulante).")
        nome = ""
        while not nome.strip(): nome = input("Qual o seu nome, Doutor(a)? ").strip()

        print_lentamente("\nEscolha sua Classe Médica:")
        for i, c_enum in enumerate(ClassesMedico):
            print_lentamente(f"{i+1}. {c_enum.value['nome']}: {c_enum.value['descricao_habilidade']} (Ex: {c_enum.value['exemplo_uso']})")
        classe_idx = solicitar_input_numerico("Sua Classe: ", 1, len(ClassesMedico))
        classe_esc = list(ClassesMedico)[classe_idx-1]

        print_lentamente("\nEscolha seu Traço de Personalidade:")
        for i, t_enum in enumerate(TracosMedico):
            ex_traco = TracosMedico[t_enum.name].value.get('exemplo', t_enum.value['descricao'])
            print_lentamente(f"{i+1}. {t_enum.value['nome']}: {ex_traco}")
        traco_idx = solicitar_input_numerico("Seu Traço: ", 1, len(TracosMedico))
        traco_esc = list(TracosMedico)[traco_idx-1]

        self.jogador = Jogador(nome, classe_esc, traco_esc)
        print_lentamente(f"\nBem-vindo(a), Dr(a). {self.jogador.nome}!")
        print_lentamente(str(self.jogador))

        combinacao = self.jogador.verificar_combinacao_especial()
        if combinacao:
            msg_combinacao = combinacao.get("interacao_unica", combinacao if isinstance(combinacao, str) else "Efeito especial ativo!")
            print_lentamente(f"\n✨ Interação Especial Classe/Traço: {msg_combinacao} ✨")
            if isinstance(combinacao, dict) and "respostas_exemplo" in combinacao:
                print_lentamente("  Exemplos de como isso pode se manifestar:")
                for tipo_resp, ex_resp in combinacao["respostas_exemplo"]:
                    print_lentamente(f"  - Se escolher {tipo_resp}: \"{ex_resp}\"")

        self.casos_concluidos_ids = []
        self.loop_principal_jogo()

    def carregar_jogo_salvo(self):
        jogador_carregado, cids = carregar_jogo()
        if jogador_carregado:
            self.jogador = jogador_carregado; self.casos_concluidos_ids = cids
            print_lentamente(f"\nBem-vindo(a) de volta, Dr(a). {self.jogador.nome}!")
            print_lentamente(str(self.jogador))
            self.loop_principal_jogo()
        else:
            print_lentamente("Falha ao carregar. Iniciando um novo jogo...")
            time.sleep(1)
            self.iniciar_novo_jogo()

    def _gerar_e_iniciar_novo_caso(self):
        if not self.gemini_api.model:
            print_lentamente("API Gemini não configurada. Impossível gerar caso.")
            return False

        print_lentamente("\nBuscando prontuário de um novo paciente misterioso...")
        temas_possiveis = [
            "Paciente com pele de cor estranha e delírios", "O Homem da Perna Invisível (dor fantasma com complicações)",
            "Acidente com Múltiplas Vítimas (foco em um caso principal, mas com menção ao caos)",
            "Paciente que Brilha no Escuro (inspirado em House S05E16)",
            "A Enfermeira que Não Sentia Dor (inspirado em House S02E12, com condição subjacente)",
            "O Homem que Mordeu o Cão (hipocondria com sintoma real leve)",
            "O Homem que Falava com Plantas (sintomas psicóticos com causa orgânica, tipo Tuberculose cerebral)",
            "Homem diz que engoliu um Bluetooth (corpo estranho + outra condição inesperada)"
        ]
        if self.jogador.casos_resolvidos < 2:
            tema_escolhido = random.choice(temas_possiveis[:4] + [None])
            dificuldade_sugerida = "fácil" if self.jogador.casos_resolvidos == 0 else "normal"
        else:
            tema_escolhido = random.choice(temas_possiveis + [None, None])
            dificuldade_sugerida = random.choice(["normal", "difícil", "bizarro" if self.jogador.fama_de_maluco > 10 else "difícil"])

        print_lentamente(f"(Sorteando caso... tema possível: '{tema_escolhido if tema_escolhido else 'Totalmente Aleatório'}', dificuldade: {dificuldade_sugerida})")
        dados_caso_json = self.gemini_api.gerar_caso_clinico(self.jogador, tema_caso=tema_escolhido, dificuldade=dificuldade_sugerida, casos_anteriores_ids=self.casos_concluidos_ids)

        if not isinstance(dados_caso_json, dict):
            print_lentamente(f"⚠️ Falha crítica ao gerar caso (não é JSON): {str(dados_caso_json)[:500]}...")
            return False

        try:
            self.caso_atual = CasoClinico(**dados_caso_json)
            self.caso_atual.jogador = self.jogador
            print_lentamente(f"\n📜 NOVO CASO: {self.caso_atual.titulo_caso} 📜")

            self.caso_atual.adicionar_progresso_cena(self.caso_atual.introducao_narrativa_imersiva)
            self._aplicar_habilidades_inicio_caso()

            self.caso_atual.progresso_narrativa_cena_atual = [self.caso_atual.introducao_narrativa_imersiva.split('\n')[-1]]
            self._atualizar_opcoes_de_cena_caso_atual()

            return True
        except TypeError as te:
            print_lentamente(f"❌ Erro ao criar CasoClinico com dados da API (TypeError): {te}")
            print_lentamente(f"Dados recebidos que causaram o erro: {json.dumps(dados_caso_json, indent=2)}")
            return False
        except KeyError as ke:
            print_lentamente(f"❌ Erro ao criar CasoClinico (Chave faltando: {ke}).")
            print_lentamente(f"Dados recebidos: {json.dumps(dados_caso_json, indent=2)}")
            return False
        except Exception as e:
            print_lentamente(f"❌ Erro inesperado ao iniciar novo caso: {type(e).__name__} - {e}")
            print_lentamente(f"Dados recebidos: {json.dumps(dados_caso_json, indent=2)}")
            return False

    def _atualizar_opcoes_de_cena_caso_atual(self): # NOVO MÉTODO
        if not self.caso_atual: return
        print_lentamente("\n(Gerando novas opções de interação para a cena...)") # Movido para antes da chamada da API
        opcoes_geradas = self.gemini_api.gerar_opcoes_de_cena(
            self.jogador,
            self.caso_atual,
            self.caso_atual.progresso_narrativo_cena_atual
        )
        if isinstance(opcoes_geradas, list) and len(opcoes_geradas) == 8 and \
           all(isinstance(opt, dict) and "tom" in opt and "opcao_texto" in opt for opt in opcoes_geradas):
            self.caso_atual.opcoes_cena_atuais = opcoes_geradas
        else:
            print_lentamente(f"⚠️ Aviso: Não foi possível gerar as 8 opções de cena formatadas corretamente.")
            print_lentamente(f"   Resposta da API: {str(opcoes_geradas)[:200]}...")
            self.caso_atual.opcoes_cena_atuais = [
                {"tom": "Pragmática", "opcao_texto": "Reavaliar os sinais vitais do paciente."},
                {"tom": "Empática", "opcao_texto": "Perguntar ao paciente como ele está se sentindo agora."},
                {"tom": "Cautelosa", "opcao_texto": "Aguardar e observar por um momento."} ,
                {"tom": "Cínica", "opcao_texto": "Perguntar se o paciente já pensou em ser ator."},
                {"tom": "Filosófica", "opcao_texto": "Refletir sobre a natureza da queixa principal."},
                {"tom": "Absurda", "opcao_texto": "Verificar se o paciente tem algum objeto amaldiçoado consigo."},
                {"tom": "Heróica", "opcao_texto": "Declarar que este caso será resolvido com brilhantismo."},
                {"tom": "Desastre Iminente", "opcao_texto": "Sugerir um tratamento experimental com sanguessugas."}
            ]
            random.shuffle(self.caso_atual.opcoes_cena_atuais)
            self.caso_atual.opcoes_cena_atuais = self.caso_atual.opcoes_cena_atuais[:8]

    def _chance_intervencao_npc(self):
        if not self.caso_atual or random.random() > 0.20:
            return

        npc_nomes = list(NPC_DATA.keys())
        npc_escolhido = random.choice(npc_nomes)

        resumo_situacao_para_npc = (
            f"O Dr(a). {self.jogador.nome} está atendendo o caso '{self.caso_atual.titulo_caso}'. "
            f"Sintomas conhecidos pelo jogador: {', '.join(list(set(self.caso_atual.sintomas_revelados_jogador)))}. "
            f"Hipóteses atuais do jogador: {', '.join(self.caso_atual.hipoteses_jogador) if self.caso_atual.hipoteses_jogador else 'nenhuma clara'}."
        )

        # Adiciona a última interação da cena para dar mais contexto ao NPC
        ultima_interacao_cena = self.caso_atual.progresso_narrativo_cena_atual[-1] if self.caso_atual.progresso_narrativo_cena_atual else "Início da cena."
        contexto_npc_completo = f"{resumo_situacao_para_npc} Interação mais recente: {ultima_interacao_cena}"


        # Não usar adicionar_progresso_cena aqui para o print inicial, pois ele já será usado para o diálogo do NPC
        print_lentamente(f"\n--- Intervenção Inesperada de {npc_escolhido}! ---")
        dialogo_npc = self.gemini_api.gerar_dialogo_npc(
            npc_escolhido,
            {"titulo_caso": self.caso_atual.titulo_caso, "resumo_situacao_para_npc": contexto_npc_completo},
            self.jogador
        )
        if "Erro" not in dialogo_npc:
            self.caso_atual.adicionar_progresso_cena(f"{npc_escolhido}: \"{dialogo_npc}\"") # Usa adicionar_progresso_cena
            if npc_escolhido == "Dr. Howser": self.caso_atual.dicas_howser_caso.append(dialogo_npc)
            elif npc_escolhido == "Dra. Meridee Grey": self.caso_atual.diario_meridee_caso.append(dialogo_npc)
        else:
            self.caso_atual.adicionar_progresso_cena(f"{npc_escolhido} parece que ia dizer algo, mas se distrai com uma mosca particularmente interessante.")

    def interagir_com_opcao_de_cena(self, opcao_dict):
        if not self.caso_atual: return

        texto_acao = opcao_dict["opcao_texto"]
        tom_acao = opcao_dict["tom"]

        self.caso_atual.adicionar_progresso_cena(f"\nVocê ({tom_acao}): \"{texto_acao}\"") # Usa adicionar_progresso_cena

        narrativa_resposta = self.gemini_api.gerar_resposta_interativa(
            self.jogador,
            self.caso_atual,
            texto_acao,
            tom_acao
        )

        if isinstance(narrativa_resposta, str) and "Erro" in narrativa_resposta:
            self.caso_atual.adicionar_progresso_cena(f"Houve um problema na resposta da IA: {narrativa_resposta}")
        else:
            self.caso_atual.adicionar_progresso_cena(narrativa_resposta)

            for sint_real in self.caso_atual.sintomas_reais_ocultos:
                if f"'{sint_real.lower()}'" in narrativa_resposta.lower() and sint_real not in self.caso_atual.sintomas_revelados_jogador:
                    self.caso_atual.sintomas_revelados_jogador.append(sint_real)
                    self.caso_atual.adicionar_progresso_cena(f"🔎 (Sintoma '{sint_real}' foi revelado/confirmado através da interação!)")

        self.caso_atual.opcoes_cena_atuais = [] # Força regenerar opções para a próxima "sub-cena"

    def conduzir_caso(self):
        if not self.caso_atual:
            print_lentamente("⚠️ Nenhum caso ativo para conduzir.")
            return

        while self.caso_atual and not self.caso_atual.resolvido:
            # Não chamar exibir_resumo_mental aqui automaticamente. Deixe como uma ação do jogador.
            print_lentamente(f"\n--- Investigando: {self.caso_atual.titulo_caso} ---")
            self._chance_intervencao_npc()

            if not self.caso_atual.opcoes_cena_atuais:
                self._atualizar_opcoes_de_cena_caso_atual()

            menu_acoes_cena = {}
            menu_acoes_fixas = {}

            print_lentamente("\nEscolha uma Ação/Diálogo para a Cena:")
            if self.caso_atual.opcoes_cena_atuais:
                for i, opcao_data in enumerate(self.caso_atual.opcoes_cena_atuais):
                    menu_acoes_cena[i + 1] = opcao_data
                    print_lentamente(f"  {i + 1}. [{opcao_data['tom']}] {opcao_data['opcao_texto']}")
            else:
                print_lentamente("  (A IA não gerou opções de cena. Verifique a API ou tente uma ação fixa.)")

            next_fixed_idx = (len(menu_acoes_cena) if menu_acoes_cena else 0) + 1

            print_lentamente("\nOutras Ações Possíveis:")
            idx_hipotese = next_fixed_idx; menu_acoes_fixas[idx_hipotese] = {"tipo": "hipotese", "texto": "Adicionar/Ver hipóteses diagnósticas"}; print_lentamente(f"  {idx_hipotese}. {menu_acoes_fixas[idx_hipotese]['texto']}"); next_fixed_idx += 1
            idx_howser = next_fixed_idx; menu_acoes_fixas[idx_howser] = {"tipo": "dica_howser", "texto": "(Modo Médico) Pedir dica ao Dr. Howser"}; print_lentamente(f"  {idx_howser}. {menu_acoes_fixas[idx_howser]['texto']}"); next_fixed_idx += 1
            idx_meridee = next_fixed_idx; menu_acoes_fixas[idx_meridee] = {"tipo": "dica_meridee", "texto": "(Modo Médico) Consultar diário da Dra. Meridee"}; print_lentamente(f"  {idx_meridee}. {menu_acoes_fixas[idx_meridee]['texto']}"); next_fixed_idx += 1

            idx_artista = -1
            if self.jogador.traco == TracosMedico.ARTISTA_FRUSTRADO:
                idx_artista = next_fixed_idx; menu_acoes_fixas[idx_artista] = {"tipo": "desenhar", "texto": "Desenhar um sintoma/observação"}; print_lentamente(f"  {idx_artista}. {menu_acoes_fixas[idx_artista]['texto']}"); next_fixed_idx += 1

            idx_resumo = next_fixed_idx; menu_acoes_fixas[idx_resumo] = {"tipo": "resumo_mental", "texto": "Organizar pensamentos / Revisar prontuário"}; print_lentamente(f"  {idx_resumo}. {menu_acoes_fixas[idx_resumo]['texto']}"); next_fixed_idx +=1

            idx_diagnostico = next_fixed_idx; menu_acoes_fixas[idx_diagnostico] = {"tipo": "diagnosticar", "texto": "TENTAR DIAGNÓSTICO FINAL"}; print_lentamente(f"  {idx_diagnostico}. {menu_acoes_fixas[idx_diagnostico]['texto']}"); next_fixed_idx += 1
            idx_salvar = next_fixed_idx; menu_acoes_fixas[idx_salvar] = {"tipo": "salvar", "texto": "Salvar e Voltar ao Menu Principal"}; print_lentamente(f"  {idx_salvar}. {menu_acoes_fixas[idx_salvar]['texto']}");

            max_total_option_val = next_fixed_idx -1 # Corrigido para pegar o último índice usado

            escolha_num_str = input(f"Sua ação (1-{max_total_option_val}): ").strip()
            if not escolha_num_str: continue
            try:
                escolha_num = int(escolha_num_str)
                if not (1 <= escolha_num <= max_total_option_val): raise ValueError
            except ValueError:
                print_lentamente("Opção inválida. Tente novamente.")
                continue

            if escolha_num in menu_acoes_cena:
                opcao_de_cena_escolhida = menu_acoes_cena[escolha_num]
                self.interagir_com_opcao_de_cena(opcao_de_cena_escolhida)
            elif escolha_num in menu_acoes_fixas:
                acao_fixa_obj = menu_acoes_fixas[escolha_num]
                tipo_acao_fixa = acao_fixa_obj["tipo"]

                if tipo_acao_fixa == "hipotese": self.listar_ou_add_hipoteses()
                elif tipo_acao_fixa == "dica_howser": self.pedir_dica_npc("Dr. Howser")
                elif tipo_acao_fixa == "dica_meridee": self.pedir_dica_npc("Dra. Meridee Grey")
                elif tipo_acao_fixa == "desenhar" and self.jogador.traco == TracosMedico.ARTISTA_FRUSTRADO: self.desenhar_sintoma()
                elif tipo_acao_fixa == "resumo_mental": self.caso_atual.exibir_resumo_mental(self.jogador.nome)
                elif tipo_acao_fixa == "diagnosticar": self.propor_diagnostico_final()
                elif tipo_acao_fixa == "salvar":
                    salvar_jogo(self.jogador, self.casos_concluidos_ids)
                    self.caso_atual = None
                    return
            else:
                print_lentamente("Opção não reconhecida.")

            if self.caso_atual and self.caso_atual.resolvido:
                # A narrativa do desfecho já foi impressa por propor_diagnostico_final
                print_lentamente("\n[CASO CONCLUÍDO]")
                self.caso_atual = None
                break

            time.sleep(0.3)

    # Manter os outros métodos como:
    # listar_ou_add_hipoteses(self) -> usar adicionar_progresso_cena se quiser que faça parte do histórico da cena
    # pedir_dica_npc(self, npc_nome_param) -> usar adicionar_progresso_cena
    # desenhar_sintoma(self) -> usar adicionar_progresso_cena
    # propor_diagnostico_final(self) -> OK
    # finalizar_caso_com_diagnostico(self, diagnostico_proposto) -> OK
    # loop_principal_jogo(self) -> OK

    def listar_ou_add_hipoteses(self):
        if not self.caso_atual: return
        print_lentamente("\n📋 Suas hipóteses diagnósticas atuais:")
        if not self.caso_atual.hipoteses_jogador: print_lentamente("(Nenhuma ainda.)")
        else:
            for i, h in enumerate(self.caso_atual.hipoteses_jogador): print_lentamente(f"  {i+1}. {h}")

        nova_h = input("Adicionar/refinar hipótese (ou Enter para voltar): ").strip()
        if nova_h:
            self.caso_atual.hipoteses_jogador.append(nova_h)
            self.caso_atual.adicionar_progresso_cena(f"Você pondera e anota uma nova suspeita: {nova_h}.")
            if "lúpus" in nova_h.lower().strip() and self.caso_atual.id_caso != "caso_secreto_do_lupus_real_007":
                self.caso_atual.adicionar_progresso_cena("Uma voz familiar ecoa em sua mente, soando suspeitamente como Dr. Howser: 'É claro que você pensou em lúpus. NUNCA é lúpus.'")
            elif "lúpus" in nova_h.lower().strip() and self.caso_atual.id_caso == "caso_secreto_do_lupus_real_007":
                 self.caso_atual.adicionar_progresso_cena("Por um instante, você quase ouve Dr. Howser engasgar. 'Espera... só desta vez... talvez.'")

            if self.jogador.traco == TracosMedico.TEORICO_CONSPIRACAO and random.random() < 0.4:
                hipotese_conspiratoria = self.gemini_api._gerar_conteudo_seguro(
                    f"Dr(a). {self.jogador.nome}, um Teórico da Conspiração, considera a hipótese '{nova_h}'. "
                    f"Gere uma hipótese alternativa BIZARRA e CONSPIRATÓRIA que ele/ela também consideraria para o caso '{self.caso_atual.titulo_caso}' "
                    f"com sintomas {self.caso_atual.sintomas_revelados_jogador}. (1 frase curta)"
                )
                if "Erro" not in hipotese_conspiratoria:
                    self.caso_atual.hipoteses_jogador.append(f"{hipotese_conspiratoria} (Conspiração)")
                    self.caso_atual.adicionar_progresso_cena(f"👽 (E se... e se '{hipotese_conspiratoria}' for a verdadeira causa? Sua mente borbulha com as possibilidades.)")
                    self.jogador.fama_de_maluco +=1

    def pedir_dica_npc(self, npc_nome_param):
        if not self.caso_atual: return
        lista_dicas_ou_reflexoes = []
        prefixo_narrativo = ""

        if npc_nome_param == "Dr. Howser":
            lista_dicas_ou_reflexoes = self.caso_atual.dicas_howser_caso
            prefixo_narrativo = "Dr. Howser resmunga do canto da sala:"
        elif npc_nome_param == "Dra. Meridee Grey":
            lista_dicas_ou_reflexoes = self.caso_atual.diario_meridee_caso
            prefixo_narrativo = "(Uma voz interior, como a da Dra. Meridee, sussurra:"

        if lista_dicas_ou_reflexoes:
            item_escolhido = random.choice(lista_dicas_ou_reflexoes)
            self.caso_atual.adicionar_progresso_cena(f"{prefixo_narrativo} \"{item_escolhido}\"")
        else:
            contexto_geral_para_dica = f"O jogador Dr(a). {self.jogador.nome} está investigando o caso '{self.caso_atual.titulo_caso}'. Sintomas atuais observados: {', '.join(list(set(self.caso_atual.sintomas_revelados_jogador)))}. O diagnóstico real do caso é {self.caso_atual.diagnostico_real_medico} (jogador não sabe)."
            nova_interacao_npc = self.gemini_api.gerar_dialogo_npc(npc_nome_param,
                {"titulo_caso": self.caso_atual.titulo_caso, "resumo_situacao_para_npc": contexto_geral_para_dica},
                self.jogador)

            if "Erro" in nova_interacao_npc:
                self.caso_atual.adicionar_progresso_cena(f"{npc_nome_param} parece estar muito ocupado para dar atenção agora. ({nova_interacao_npc})")
            else:
                if npc_nome_param == "Dr. Howser": self.caso_atual.dicas_howser_caso.append(nova_interacao_npc)
                elif npc_nome_param == "Dra. Meridee Grey": self.caso_atual.diario_meridee_caso.append(nova_interacao_npc)
                self.caso_atual.adicionar_progresso_cena(f"{npc_nome_param} aparece de repente e comenta: \"{nova_interacao_npc}\"")

    def desenhar_sintoma(self):
        if not self.caso_atual: return
        if self.jogador.traco != TracosMedico.ARTISTA_FRUSTRADO:
            # Esta mensagem não deve ser adicionada ao progresso da cena se o jogador não pode fazer
            print_lentamente("Apenas Artistas Frustrados podem usar esta nobre forma de diagnóstico.")
            return

        sint_desenhar = input("Qual sintoma ou observação você quer imortalizar em sua 'arte'?\n> ").strip()
        if not sint_desenhar:
            self.caso_atual.adicionar_progresso_cena("A inspiração artística não veio desta vez.")
            return

        desc_desenho_gerada = self.gemini_api.gerar_imagem_sintoma_descricao_textual(self.jogador, sint_desenhar)
        self.caso_atual.adicionar_progresso_cena(f"\n--- 🎨 Obra Prima Incompreendida de Dr(a). {self.jogador.nome} 🎨 ---")
        if "Erro" in desc_desenho_gerada:
            self.caso_atual.adicionar_progresso_cena(f"Sua tentativa de desenhar foi... um borrão. (Erro da API: {desc_desenho_gerada})")
        else:
            self.caso_atual.adicionar_progresso_cena(desc_desenho_gerada)
            self.caso_atual.adicionar_progresso_cena("(Como o hospital não tem verba para impressoras coloridas nos anos 80, você descreve sua visão gloriosa aos colegas menos iluminados, esperando que entendam sua genialidade).")
            self.jogador.fama_de_maluco += random.randint(0,1)

    def propor_diagnostico_final(self):
        if not self.caso_atual: return

        if not self.caso_atual.opcoes_diagnostico_atuais:
            self._atualizar_opcoes_diagnostico_caso_atual()

        if not self.caso_atual.opcoes_diagnostico_atuais: # Se ainda falhou
            self.caso_atual.adicionar_progresso_cena("O sistema de classificação de doenças parece estar em manutenção. Impossível listar opções de diagnóstico agora.")
            diag_proposto_texto = input(f"FALLBACK: Qual o seu diagnóstico final para {self.caso_atual.paciente_info.get('nome', 'este paciente')}?\n> ").strip()
            if not diag_proposto_texto:
                 self.caso_atual.adicionar_progresso_cena("Você decide esperar mais um pouco.")
                 return
            self.finalizar_caso_com_diagnostico(diag_proposto_texto)
            return

        print_lentamente("\nEscolha o Diagnóstico Final:")
        for i, diag_opcao in enumerate(self.caso_atual.opcoes_diagnostico_atuais):
            print_lentamente(f"  {i+1}. {diag_opcao}")

        escolha_idx_str = input(f"Seu diagnóstico (1-{len(self.caso_atual.opcoes_diagnostico_atuais)}): ").strip()
        try:
            escolha_idx = int(escolha_idx_str) -1
            if not (0 <= escolha_idx < len(self.caso_atual.opcoes_diagnostico_atuais)):
                raise ValueError
            diagnostico_escolhido = self.caso_atual.opcoes_diagnostico_atuais[escolha_idx]
        except (ValueError, IndexError):
            self.caso_atual.adicionar_progresso_cena("Escolha inválida. Você decide ponderar mais.")
            return

        self.finalizar_caso_com_diagnostico(diagnostico_escolhido)

    def finalizar_caso_com_diagnostico(self, diagnostico_proposto):
        if not self.caso_atual: return
        self.caso_atual.adicionar_progresso_cena(f"\nVocê declara com (incerta?) convicção: \"O diagnóstico é... {diagnostico_proposto}!\"")
        # print_lentamente(f"\nProcessando seu veredito: '{diagnostico_proposto}'...") # Removido, já faz parte do adicionar_progresso_cena
        desfecho_obj_json = self.gemini_api.gerar_desfecho_caso(self.jogador, self.caso_atual, diagnostico_proposto)

        if not isinstance(desfecho_obj_json, dict):
            print_lentamente(f"⚠️ Falha crítica ao gerar desfecho (não é JSON válido): {str(desfecho_obj_json)[:500]}...")
            self.caso_atual.adicionar_progresso_cena("O sistema de prontuários explodiu (metaforicamente). O destino do paciente é um mistério cósmico.")
            self.caso_atual.resolvido = True
            self.jogador.reputacao = max(0, self.jogador.reputacao - 2)
            return

        try:
            # Usar adicionar_progresso_cena para a narrativa do desfecho para manter consistência
            self.caso_atual.adicionar_progresso_cena("\n--- 🩺 DESFECHO DO CASO 🩺 ---")
            self.caso_atual.adicionar_progresso_cena(desfecho_obj_json["desfecho_narrativa"])
            self.caso_atual.resolvido = True
            self.caso_atual.desfecho_final = desfecho_obj_json

            self.jogador.reputacao = max(0, min(100, self.jogador.reputacao + desfecho_obj_json.get("reputacao_delta", 0)))
            self.jogador.pontuacao_diagnostica += desfecho_obj_json.get("pontuacao_diagnostica_delta", 0)
            self.jogador.fama_de_maluco = max(0, self.jogador.fama_de_maluco + desfecho_obj_json.get("fama_de_maluco_delta", 0))
            self.jogador.casos_resolvidos += 1
            if self.caso_atual.id_caso not in self.casos_concluidos_ids:
                 self.casos_concluidos_ids.append(self.caso_atual.id_caso)

            # A Ficha Educativa é mais um relatório, então print_lentamente direto é ok aqui.
            print_lentamente(f"\n--- {desfecho_obj_json.get('ficha_educativa_titulo','Ficha Educativa do Caso')} ---")
            print_lentamente(f"Seu Diagnóstico: {desfecho_obj_json['diagnostico_final_jogador']}")
            print_lentamente(f"Diagnóstico Correto: {desfecho_obj_json['diagnostico_correto_caso']}")
            print_lentamente(f"Você acertou? {'Sim!' if desfecho_obj_json['acertou'] else 'Não desta vez.'}")
            print_lentamente(f"\nDoença Real: {desfecho_obj_json['doenca_real_ficha']}")
            print_lentamente(f"Explicação Simplificada: {desfecho_obj_json['explicacao_doenca_ficha']}")
            print_lentamente(f"Erro Comum: {desfecho_obj_json['erro_comum_ficha']}")
            print_lentamente(f"Dica para Vida Real: {desfecho_obj_json['dica_vida_real_ficha']}")

            if desfecho_obj_json.get("comentario_howser_final"):
                print_lentamente(f"Dr. Howser comenta sarcasticamente: \"{desfecho_obj_json['comentario_howser_final']}\"")
            if desfecho_obj_json.get("comentario_meridee_final"):
                print_lentamente(f"Dra. Meridee reflete poeticamente: \"{desfecho_obj_json['comentario_meridee_final']}\"")

            print_lentamente(f"\nSeu status atual: Reputação {self.jogador.reputacao}, Pontos de Diagnóstico {self.jogador.pontuacao_diagnostica}, Fama de Maluco {self.jogador.fama_de_maluco}")

        except KeyError as e:
            print_lentamente(f"❌ Erro ao processar dados do desfecho (chave faltando: {e}).")
            print_lentamente(f"Dados recebidos do desfecho: {json.dumps(desfecho_obj_json, indent=2)}")
            self.caso_atual.adicionar_progresso_cena("Um erro bizarro ocorreu nos arquivos do hospital. O desfecho é... nebuloso.")
        except Exception as e:
            print_lentamente(f"❌ Erro inesperado ao finalizar o caso: {type(e).__name__} - {e}")
            self.caso_atual.adicionar_progresso_cena("O hospital está um caos! Algo deu muito errado com a papelada.")

    def loop_principal_jogo(self):
        if not self.jogador:
            print_lentamente("Erro crítico: Jogador não foi inicializado. Saindo.")
            return

        rodando = True
        while rodando:
            print_lentamente(f"\n🏥 Hospital Anos 80 - Dr(a). {self.jogador.nome} 🏥")
            print_lentamente(f"Rep: {self.jogador.reputacao} | Casos Resolvidos: {self.jogador.casos_resolvidos} | Diag. Certos: {self.jogador.pontuacao_diagnostica} | Fama de Maluco: {self.jogador.fama_de_maluco}")
            print_lentamente("--- Menu Principal do Plantão ---")
            print_lentamente("1. Atender próximo paciente (Novo Caso Clínico)")
            print_lentamente("2. Ver Status Completo do Médico")
            print_lentamente("3. Salvar Jogo (Progresso do Jogador)")
            print_lentamente("4. Sair do Plantão (Encerrar Jogo)")

            escolha_menu_str = input(f"Sua escolha (1-4): ").strip()
            if not escolha_menu_str: continue

            try:
                escolha_menu = int(escolha_menu_str)
                if not (1 <= escolha_menu <= 4): raise ValueError
            except ValueError:
                print_lentamente("Opção inválida. Tente novamente.")
                continue

            if escolha_menu == 1:
                if self._gerar_e_iniciar_novo_caso():
                    self.conduzir_caso()
                    if self.caso_atual is None:
                        print_lentamente("\nRetornando ao saguão principal do hospital...")
                else:
                    print_lentamente("⚠️ Não foi possível iniciar um novo caso. Verifique a API ou tente mais tarde.")
            elif escolha_menu == 2:
                print_lentamente("\n--- Status Completo do Médico ---")
                print_lentamente(str(self.jogador))
            elif escolha_menu == 3:
                salvar_jogo(self.jogador, self.casos_concluidos_ids)
            elif escolha_menu == 4:
                print_lentamente("Obrigado por jogar 'Plantão 24h: O Mistério do Paciente X'! Até o próximo (caótico) plantão.")
                rodando = False

            if self.jogador.reputacao <= 0:
                print_lentamente("\n--- ☠️ FIM DE JOGO POR REPUTAÇÃO NULA ☠️ ---")
                # ... (manter mensagens de game over)
                rodando = False

            if self.jogador.fama_de_maluco >= 20 and random.random() < (0.1 * (self.jogador.fama_de_maluco - 19)):
                print_lentamente("\n--- 🤪 FIM DE JOGO POR LOUCURA EXCESSIVA 🤪 ---")
                # ... (manter mensagens de game over)
                rodando = False
        print_lentamente("Fim da simulação.")


print("✅ Classes e funções do jogo (incluindo GeminiAPI e CasoClinico modificadas) definidas.")

In [16]:
if __name__ == "__main__" and '__file__' not in globals(): # Condição para rodar no Colab
    if generative_model is None:
        print_lentamente("❌ O jogo não pode iniciar sem a configuração da API Gemini.")
        print_lentamente("Verifique a Célula 2 e se você configurou o Secret 'GOOGLE_API_KEY' corretamente.")
    else:
        jogo_rpg_medico = JogoMedicoRPG()
        print_lentamente("\n--- 🚑 Plantão 24h: O Mistério do Paciente X (Protótipo Colab) 🚑 ---")
        print_lentamente("1. Iniciar Novo Jogo")
        print_lentamente("2. Carregar Jogo Salvo (se houver)")
        print_lentamente("3. Sair")

        escolha_inicial_jogo = None
        while escolha_inicial_jogo is None:
            escolha_inicial_jogo_str = input("Escolha uma opção (1-3): ").strip()
            if not escolha_inicial_jogo_str: continue # Ignora Enter
            try:
                escolha_inicial_jogo = int(escolha_inicial_jogo_str)
                if not (1 <= escolha_inicial_jogo <= 3):
                    print_lentamente("Opção inválida.")
                    escolha_inicial_jogo = None # Força nova tentativa
            except ValueError:
                print_lentamente("Entrada numérica inválida.")
                escolha_inicial_jogo = None


        if escolha_inicial_jogo == 1:
            jogo_rpg_medico.iniciar_novo_jogo()
        elif escolha_inicial_jogo == 2:
            jogo_rpg_medico.carregar_jogo_salvo()
        else: # Inclui escolha 3
            print_lentamente("Até a próxima!")

KeyboardInterrupt: Interrupted by user